In this code we will fetch the missing datas about the stock like importing the Sectors and Industry of the stock they are trading, buy and sell time and date. Preprocess the data, perform Data Cleaning and at last will do Exploratory Data Analysis to find the hidden patterns and trends in the data.

Now we will import the necessary libaries.

In [ ]:
import pandas as pd
import numpy as np  
import yfinance as yf
from datetime import datetime  
import matplotlib.pyplot as plt
import seaborn as sns

Now we will load the Dataset to Preprocess.

In [ ]:
# Load datasets
pnl = pd.read_excel(r'Stocks_PnL_3788142010_01-10-2021_11-10-2025_report.xlsx')
order_history = pd.read_excel(r'Stocks_Order_History_3788142010_01-10-2021_11-10-2025.xlsx')

Now let's find some information about the data

In [ ]:
pnl.info()
pnl.shape
pnl.describe()

similarly for the Order's history

In [ ]:
order_history.info()
order_history.shape
order_history.describe()

Now Let's see how the data looks 

In [ ]:
pnl.head()

Similarly for the Order history

In [ ]:
order_history.head()

Now lets fetch the execution date and time for both the buy and sell orders from the order history to the pnl to do the further preprocesses and find the time-based analysis. 

In [ ]:
# Additional Code to Split 'Execution Date and time' into Separate Columns

# 1. Convert 'Execution date and time' to datetime
order_history['Execution DateTime'] = pd.to_datetime(order_history['Execution date and time'])

# 2. Extract Date and Time into separate columns
order_history['Execution Date'] = order_history['Execution DateTime'].dt.date  # Only date
order_history['Execution Time'] = order_history['Execution DateTime'].dt.time  # Only time

# 3. Drop the original column (optional)
order_history.drop('Execution date and time', axis=1, inplace=True)

# 4. Save updated file
order_history.to_excel('Stocks_Order_History_3788142010_01-10-2021_11-10-2025.xlsx', index=False)

print("Execution Date & Time alag ho gaya! Check 'Stocks_Order_History_3788142010_01-10-2021_11-10-2025.xlsx'")

# Convert to datetime
pnl['Buy date'] = pd.to_datetime(pnl['Buy date']).dt.date
pnl['Sell date'] = pd.to_datetime(pnl['Sell date']).dt.date

# Function to validate market hours (9:15 AM to 3:30 PM)
def is_market_time(dt):
    if not pd.isna(dt):
        time = dt.time()
        return (time >= pd.Timestamp('09:15:00').time()) & (time <= pd.Timestamp('15:30:00').time())
    return False

# Match Buy/Sell Times from Order History
def get_times_from_order_history(pnl_row, order_df):
    stock_name = pnl_row['Stock name']
    isin = pnl_row['ISIN']
    quantity = pnl_row['Quantity']
    buy_date = pnl_row['Buy date']
    sell_date = pnl_row['Sell date']
    
    # Filter relevant orders
    stock_orders = order_df[
        (order_df['Stock name'] == stock_name) & 
        (order_df['ISIN'] == isin)
    ].copy()
    
    # Case 1: Intraday (Buy & Sell same day)
    if buy_date == sell_date:
        buy_order = stock_orders[
            (stock_orders['Execution DateTime'].dt.date == buy_date) &
            (stock_orders['Type'] == 'BUY') &
            (stock_orders['Execution DateTime'].apply(is_market_time))
        ].sort_values('Execution DateTime').head(1)
        
        sell_order = stock_orders[
            (stock_orders['Execution DateTime'].dt.date == sell_date) &
            (stock_orders['Type'] == 'SELL') &
            (stock_orders['Execution DateTime'].apply(is_market_time))
        ].sort_values('Execution DateTime').tail(1)
    
    # Case 2: Delivery (Different dates)
    else:
        buy_order = stock_orders[
            (stock_orders['Execution DateTime'].dt.date == buy_date) &
            (stock_orders['Type'] == 'BUY') &
            (stock_orders['Execution DateTime'].apply(is_market_time))
        ].sort_values('Execution DateTime').head(1)
        
        sell_order = stock_orders[
            (stock_orders['Execution DateTime'].dt.date == sell_date) &
            (stock_orders['Type'] == 'SELL') &
            (stock_orders['Execution DateTime'].apply(is_market_time))
        ].sort_values('Execution DateTime').tail(1)
    
    # Extract times
    buy_time = buy_order['Execution DateTime'].iloc[0].time() if not buy_order.empty else pd.Timestamp('09:15:00').time()
    sell_time = sell_order['Execution DateTime'].iloc[0].time() if not sell_order.empty else pd.Timestamp('15:30:00').time()
    
    return buy_time, sell_time

# Apply to PnL
pnl[['Buy Time', 'Sell Time']] = pnl.apply(
    lambda row: pd.Series(get_times_from_order_history(row, order_history)), 
    axis=1
)

order_history.drop('Execution DateTime', axis=1, inplace=True)

# Save
pnl.to_excel('Stocks_PnL_3788142010_01-10-2021_11-10-2025_report.xlsx', index=False)
print("Done! File saved with accurate buy/sell times (9:15 AM - 3:30 PM).")


Similarly, now let's fetch the sectors and industry for each of the trades to find the sector-based and industry-based trends and patterns in the user trade history 

In [ ]:
# Create symbol mapping from order history (Symbol to Stock name)
symbol_to_name = dict(zip(order_history['Symbol'], order_history['Stock name']))
name_to_symbol = {v: k for k, v in symbol_to_name.items()}

# Get unique symbols from order history
unique_symbols = order_history['Symbol'].unique()

# Fetch sector data
sector_data = {}
for symbol in unique_symbols:
    try:
        ticker = yf.Ticker(f"{symbol}.NS")
        info = ticker.info
        sector_data[symbol] = (
            info.get('sector', 'Unknown'),
            info.get('industry', 'Unknown')
        )
    except Exception as e:
        print(f"Error fetching {symbol}: {str(e)}")
        sector_data[symbol] = ('Unknown', 'Unknown')

# Add to order history
order_history['Sector'] = order_history['Symbol'].map(lambda x: sector_data.get(x, ('Unknown', 'Unknown'))[0])
order_history['Industry'] = order_history['Symbol'].map(lambda x: sector_data.get(x, ('Unknown', 'Unknown'))[1])

# Add to PnL - first extract symbol from stock name
pnl['Symbol'] = pnl['Stock name'].map(name_to_symbol)
pnl['Sector'] = pnl['Symbol'].map(lambda x: sector_data.get(x, ('Unknown', 'Unknown'))[0])
pnl['Industry'] = pnl['Symbol'].map(lambda x: sector_data.get(x, ('Unknown', 'Unknown'))[1])

# Save files
order_history.to_excel('Stocks_Order_History_3788142010_01-10-2021_11-10-2025.xlsx', index=False)
pnl.to_excel('Stocks_PnL_3788142010_01-10-2021_11-10-2025_report.xlsx', index=False)

print("Sector and Industry data added successfully!")

Since we imported the necessary Timing and Sectorial datas for the trades, now we will be moving towards the Data Cleaning part and afterwards we will apply some EDA techniques to find some hidden patterns.  

Data Cleaning

In [ ]:
#Let's check the information of the datasets again after the preprocessing
pnl.info()
pnl.shape
pnl.describe()

Since the realised profit and loss is not calculating the charges, taxes and brokerages paid by the user to the Broker and Exchange so we will be calculating them. 

In [ ]:
# Lets Calculate the Charges, Taxes and Brokerages for each of the trades.
# Calculating the Brokerage
pnl['Brokerage'] = (pnl['Buy value'] + pnl['Sell value']) * 0.0005
pnl['Brokerage'] = pnl['Brokerage'].apply(lambda x: min(x, 20)) *(1.18)

# Calculating the Exchange Charges, SEBI Charges and Investor Protection Fund
pnl['Exchange Charges'] = (pnl['Buy value'] + pnl['Sell value']) * 0.0000297 *(1.18)
pnl['SEBI Charges'] = (pnl['Buy value'] + pnl['Sell value']) * 0.000001 *(1.18)
pnl['Investor Protection Fund'] = (pnl['Buy value'] + pnl['Sell value']) * 0.000001 *(1.18)

# Initialize columns
pnl['STT'] = 0
pnl['Stamp Duty'] = 0

# For Intraday
pnl.loc[pnl['Remark'] == 'Intraday', 'STT'] = pnl['Sell value'] * 0.00025  *(1.18)
pnl.loc[pnl['Remark'] == 'Intraday', 'Stamp Duty'] = pnl['Buy value'] * 0.00003 *(1.18)
pnl.loc[pnl['Remark'] == 'Intraday', 'Demat Charges'] = 0 *(1.18)

# For Delivery (or others)
pnl.loc[pnl['Remark'] != 'Intraday', 'STT'] = (pnl['Sell value'] + pnl['Buy value']) * 0.001 *(1.18)
pnl.loc[pnl['Remark'] != 'Intraday', 'Stamp Duty'] = pnl['Buy value'] * 0.00015 *(1.18)
pnl.loc[pnl['Remark'] != 'Intraday', 'Demat Charges'] = 13.5 *(1.18)

# For IPO
pnl.loc[pnl['Remark'] == 'New shares credit from IPO', 'STT'] = (pnl['Sell value']) * 0.001 *(1.18)
pnl.loc[pnl['Remark'] == 'New shares credit from IPO', 'Stamp Duty'] = 0 *(1.18)
pnl.loc[pnl['Remark'] == 'New shares credit from IPO', 'Exchange Charges'] = (pnl['Sell value']) * 0.0000297 *(1.18)
pnl.loc[pnl['Remark'] == 'New shares credit from IPO', 'SEBI Charges'] = (pnl['Sell value']) * 0.000001 *(1.18)
pnl.loc[pnl['Remark'] == 'New shares credit from IPO', 'Investor Protection Fund'] = (pnl['Sell value']) * 0.000001 *(1.18)

# Calculating the Total Charges including GST
pnl['Total Charges'] = (pnl['Brokerage'] + pnl['STT'] + pnl['Exchange Charges'] + pnl['SEBI Charges'] + pnl['Investor Protection Fund'] + pnl['Stamp Duty'] + pnl['Demat Charges'])

# Since we have calculated the charges, taxes and brokerages for each of the trades, we can now drop the unnecessary columns.
charges_to_drop = ['STT', 'Exchange Charges', 'SEBI Charges', 'Investor Protection Fund', 'Stamp Duty', 'Demat Charges']
pnl = pnl.drop(columns=charges_to_drop)

# Adding a new column for Total charges in the PnL dataset
pnl['Total Charges'] = pnl['Total Charges'].round(4)
pnl.to_excel('Stocks_PnL_3788142010_01-10-2021_11-10-2025_report.xlsx', index=False)

Now, let's find the Gross and Net pnl and pnl % to better understand the profits and losses. This will give the idea of overall profits user generates. 

In [ ]:
# Calculating the Gross and Net pnl and pnl %
pnl['Gross Realised P&L'] = pnl['Sell value'] - pnl['Buy value']
pnl['Gross Realised PnL %'] = (pnl['Gross Realised P&L'] / pnl['Buy value']) * 100
pnl['Net Realised P&L'] = pnl['Gross Realised P&L'] - pnl['Total Charges']
pnl['Net Realised PnL %'] = (pnl['Net Realised P&L'] / pnl['Buy value']) * 100
pnl = pnl.drop(columns= 'Realised P&L')
pnl.to_excel('Stocks_PnL_3788142010_01-10-2021_11-10-2025_report.xlsx', index=False)

Now, let's find the holding period to understand the user's preference and find the most profitable number of days for the user.

In [ ]:
# Convert dates to datetime
pnl['Buy date'] = pd.to_datetime(pnl['Buy date'], dayfirst=True)
pnl['Sell date'] = pd.to_datetime(pnl['Sell date'], dayfirst=True)

# 1. Calculate Holding Period (in days)
pnl['Holding Period (Days)'] = (pnl['Sell date'] - pnl['Buy date']).dt.days
pnl.to_excel('Stocks_PnL_3788142010_01-10-2021_11-10-2025_report.xlsx', index=False)

Now, let's find out if any missing values are there

In [ ]:
pnl.isnull().sum()
pnl.info()
pnl.describe()

Since Remarks have so many missing values, lets fill them

In [ ]:
pnl['Remark'].value_counts()
pnl['Remark'] = pnl['Remark'].fillna('Regular trade')
pnl['Remark'].value_counts()